In [23]:
from matplotlib import pyplot as plt
import numpy as np
from typing import Iterable, Dict

from comparison_testing import get_circuits
from r1cs_scripts.read_r1cs import parse_r1cs
from r1cs_scripts.circuit_representation import Circuit
from r1cs_scripts.constraint import Constraint
from normalisation import r1cs_norm
from structural_analysis.graph_clustering.signal_equivalence_clustering import naive_removal_clustering
from structural_analysis.signal_graph import shared_constraint_graph

def getvars(con) -> set:
    return set(con.A.keys()).union(con.B.keys()).union(con.C.keys()).difference(set([0]))

def count_ints(lints : Iterable[int]) -> Dict[int, int]:
    res = {}
    for i in lints:
        res[i] = res.setdefault(i, 0) + 1
    return sorted(res.items())

def is_signal_equivalence_constraint(con: Constraint) -> bool:
        return len(con.A) + len(con.B) == 0 and len(con.C) == 2 and sorted(r1cs_norm(con)[0].C.values()) == [1, con.p - 1]

files = ["Reveal", "Poseidon", "Move", "Biomebase"]
ver = "O1"

files = list(map(lambda name : "r1cs_files/" + name + ver + ".r1cs", files))
circuits = [Circuit() for _ in files]

for circ, file in zip(circuits, files):
    parse_r1cs(file, circ)

graphs = [shared_constraint_graph(circ.constraints) for circ in circuits]

In [37]:
import networkx as nx
from structural_analysis.graph_clustering.degree_clustering import _signal_data_from_cons_list
data = {}

# All shortest paths is too slow

for file, circ, g in zip(files, circuits, graphs):
    print(file)
    degree_to_signal, signal_to_cons = _signal_data_from_cons_list(circ.constraints)

    max_degree = max(degree_to_signal.keys())
    signal = degree_to_signal[max_degree][0]

    neighbourhood = set([])
    for coni in signal_to_cons[signal]:
        neighbourhood.update(getvars(circ.constraints[coni]))
    neighbourhood.remove(signal)
    
    G = g.copy()
    G.remove_node(signal)

    data[file] = 0

for file, circ in zip(files, circuits):
    print(file, data[file])


r1cs_files/RevealO1.r1cs
set()


KeyboardInterrupt: 